# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.

## Project Workspace - Machine Learning Pipeline
For the machine learning portion, you will:
1. Split the data into a training set and a test set. 
2. Then, you will create a machine learning pipeline that uses: 
    - NLTK, 
    - Scikit-learn's Pipeline and GridSearchCV 
    - to output a final model that uses the message column to predict classifications for 36 categories (multi-output classification).
3. Finally, you will export your model to a pickle file. After completing the notebook, you'll need to include your final machine learning code in ```train_classifier.py```.

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahristian001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahristian001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ahristian001\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# import libraries
import sqlite3
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

pd.set_option('max_rows', 4000)
pd.set_option('max_columns', 4000)
pd.set_option('max_colwidth', 1000)
pd.set_option('max_info_columns', 4000)
pd.set_option('max_info_rows', 4000)
pd.set_option('max_seq_items', 4000)
pd.set_option('show_dimensions', True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [9]:
def load_data(data_file):
    '''
    INPUT
        data_file - database filename
    OUTPUT
        X - input variables
        y - target variables
        category_names - message types
    '''
    
    # load data from database
    engine = create_engine(f'sqlite:///{data_file}')
    # load to database
    df = pd.read_sql("SELECT * FROM Message", engine)
    
    # define features and label arrays
    category_names = list(df.columns[4:])
    X = df['message'].values
    y = df[category_names].values
    
    return X, y, category_names

In [10]:
X, Y, category_names = load_data('DisasterResponse.db')

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    '''
    INPUT
        text - one text message
    OUTPUT
        clean_tokens - clean normalised, tokenised and lemmatised text
    '''
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    tokens = word_tokenize(text.strip())
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []

    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [123]:
def display_results(y_true, y_pred):
    '''
    INPUT
        y_true - test labels
        y_pred - predicted labels
    '''
    results_dict = dict()

    for index, col in enumerate(cols):

        # get model results as dictionary
        report = classification_report(y_true[:, index], y_pred[:, index], zero_division=0, output_dict=True)
        # get results from classification report
        accuracy = report['accuracy'] 
        f1_score = report['weighted avg']['f1-score'] 
        precision = report['weighted avg']['precision'] 
        recall = report['weighted avg']['recall']
        # set result in dictionary
        results_dict[col] = {
                'Accuracy': accuracy,
                'F1-score': f1_score,
                'Precision': precision,
                'Recall': recall
        }

    results = pd.DataFrame.from_dict(results_dict, orient='index')
    print('Aggregated results: ')
    print(results.mean().to_string())
    display(results) 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline_1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_1.fit(X_train, y_train)

Wall time: 4min 54s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F8201D84C0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
def evaluate_model(model, X_test, Y_test, category_names):
    '''
    INPUT
        model - trained model
        X_test - inbut test variabels
        Y_test - test labels
        category_names - message types
    '''
    # predict on test data
    Y_pred = model.predict(X_test)
    
    results_dict = dict()

    for index, col in enumerate(category_names):

        # get model results as dictionary
        report = classification_report(Y_test[:, index], Y_pred[:, index], zero_division=0, output_dict=True)
        # get results from classification report
        accuracy = report['accuracy'] 
        f1_score = report['weighted avg']['f1-score'] 
        precision = report['weighted avg']['precision'] 
        recall = report['weighted avg']['recall']
        # set result in dictionary
        results_dict[col] = {
                'Accuracy': accuracy,
                'F1-score': f1_score,
                'Precision': precision,
                'Recall': recall
        }

    results = pd.DataFrame.from_dict(results_dict, orient='index')
    print('Aggregated results: ')
    print(results.mean().to_string())
    display(results)

In [18]:
# predict on test data
evaluate_model(pipeline_1, X_test, y_test, category_names)

Aggregated results: 
Accuracy    0.948
F1-score    0.935
Precision   0.938
Recall      0.948


,Accuracy,F1-score,Precision,Recall
related,0.805,0.771,0.798,0.805
request,0.898,0.884,0.896,0.898
offer,0.995,0.993,0.991,0.995
aid_related,0.778,0.774,0.777,0.778
medical_help,0.926,0.897,0.909,0.926
medical_products,0.955,0.937,0.945,0.955
search_and_rescue,0.973,0.962,0.965,0.973
security,0.980,0.971,0.961,0.980
military,0.968,0.953,0.956,0.968
child_alone,1.000,1.000,1.000,1.000


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline_1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001A229D5C4C0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001A229D5C4C0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [35]:
parameters = {
    'clf__estimator__max_depth': [5, 10, 15],
    'clf__estimator__n_jobs':[ -1],
    'clf__estimator__random_state': [17],
    'clf__estimator__n_estimators': np.linspace(100, 500, 5, dtype=int),
    'clf__estimator__min_samples_split': [2, 3, 4], #list(range(1, 8))
    #'clf__estimator__verbose': [0]
}

cv = GridSearchCV(pipeline_1, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
%%time
cv.fit(X_train, y_train)
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__max_depth': 5, 'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100, 'clf__estimator__n_jobs': -1, 'clf__estimator__random_state': 17}
Wall time: 2h 24min 46s


In [207]:
evaluate_model(cv, X_test, y_test, category_names)

Aggregated results: 
Accuracy    0.928
F1-score    0.896
Precision   0.893
Recall      0.928


,Accuracy,F1-score,Precision,Recall
related,0.761,0.658,0.579,0.761
request,0.831,0.754,0.690,0.831
offer,0.995,0.993,0.991,0.995
aid_related,0.594,0.448,0.747,0.594
medical_help,0.919,0.881,0.926,0.919
medical_products,0.949,0.924,0.900,0.949
search_and_rescue,0.972,0.958,0.945,0.972
security,0.981,0.972,0.963,0.981
military,0.967,0.951,0.935,0.967
child_alone,1.000,1.000,1.000,1.000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
%%time
from sklearn.base import BaseEstimator, TransformerMixin

class TextLengthExtractor(BaseEstimator, TransformerMixin):
    '''
    A class to extend Sklearn's transformers to add the length of the text to the pipeline
    '''
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """
        Given an input text, return its length
        """
        return pd.Series(X).apply(lambda x: len(x)).values.reshape(-1, 1)
    
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore')

pipeline = Pipeline([
                        ('features', FeatureUnion([
                            ('txt_pipeline', Pipeline([
                                ('vect', CountVectorizer()),
                                ('tfidf', TfidfTransformer())
                            ])),
                            ('txt_len', TextLengthExtractor())
                        ])),
                        ('clf', MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))) 
                    ])

pipeline.fit(X_train, y_train)

Wall time: 2min 54s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('txt_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer()),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('txt_len',
                                                 TextLengthExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
     

Here, although I have used XGBoost, in the web app, I have used RandomForestClassifier. Please, feel free to try this model, as well.

In [31]:
# # predict and evaluate model on test data
evaluate_model(pipeline, X_test, y_test, category_names)

Aggregated results: 
Accuracy    0.952
F1-score    0.945
Precision   0.945
Recall      0.952


,Accuracy,F1-score,Precision,Recall
related,0.817,0.800,0.804,0.817
request,0.908,0.903,0.903,0.908
offer,0.995,0.993,0.991,0.995
aid_related,0.774,0.771,0.773,0.774
medical_help,0.927,0.916,0.912,0.927
medical_products,0.959,0.953,0.952,0.959
search_and_rescue,0.973,0.967,0.965,0.973
security,0.980,0.971,0.968,0.980
military,0.970,0.966,0.964,0.970
child_alone,1.000,1.000,1.000,1.000


### 9. Export your model as a pickle file

In [33]:
import pickle

pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.